In [56]:
%matplotlib inline
import os
import json
from collections import defaultdict
import operator
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import time
import json
from scipy import spatial
import pandas as pd
import sys
import joblib

In [57]:
n_tags = 500 #Number of tags

In [58]:
#Create tag dict
tags = defaultdict(int)
with open("lastfm_unique_tags.txt") as tagfile:
    for x in range(n_tags):
        tag = tagfile.readline().split("\t")
        tags[tag[0]] = int(tag[1][0:-1])

In [59]:

sorted_tags = sorted(tags.items(), key=operator.itemgetter(1), reverse=True)
indexed_tags = [(t[1][0], t[0]) for t in enumerate(sorted_tags[:n_tags])]

In [60]:
#Create song vector
song_vector = defaultdict(int)
for root, subdirs, files in os.walk("data/lastfm_test/"):
    for file in files:
        with open(os.path.join(root, file)) as file_data:
            file_data = json.load(file_data)
            vector = []
            file_tags = dict(file_data['tags'])
            for tag in indexed_tags:
                if tag[0] in file_tags:
                    vector.append(int(file_tags[tag[0]]))
                else:
                    vector.append(0)
            name = file_data['artist']+' - '+file_data['title']
            song_vector[name] = vector

In [8]:
#print(list(song_vector.values())[3])

song_vec = [] #Vector with song IDs
tag_vec = [] #Vector with tags

# Remove all items with fewer tags than 10 
for song in song_vector.items():
    if song[1].count(0)<490:
        song_vec.append(song[0])
        tag_vec.append(song[1])

#Number of songs in final data
print(len(song_vec)) 

#Create spatial tree to calculate find nearest vectors
tree = spatial.KDTree(tag_vec)

6817


In [9]:
# Create sim_vec by querying tree, print time
start = time.time() #Time
sim_vec = [tree.query(list(tags),10) for tags in tag_vec] #Similarity vector
end = time.time() #Time
print(end - start)

1306.3692181110382


In [75]:
#Save initial, uncomment to save temp files:
#joblib.dump(sim_vec2,'data/sim_vec')
#joblib.dump(song_vec,'data/song_vec')
#joblib.dump(tag_vec,'data/tag_vec')

# Change similars and distances to tuple pairs instead a tuple of two lists
sim_vec2 = [[(float(h),int(tup[1][i])) for i,h in enumerate(tup[0])] for tup in sim_vec]

# Create json object for export
output = []
for i,e in enumerate(song_vec):
    obj = defaultdict()
    obj['id']=e
    obj['similars']=sim_vec2[i]
    output.append(obj)

# Export json object
with open('output.json', 'w') as outfile:
    json.dump(output, outfile)

print(type(sim_vec2[0][0][0]))

<class 'float'>
